In [ ]:
from functools import partial
import numpy as np
import h5py
from unyt import Msun, Myr, Angstrom, erg, s, Hz, cm
from astropy.cosmology import LambdaCDM
import pandas as pd
from schwimmbad import MultiPool
from uncertainties import unumpy
from scipy.stats import gaussian_kde
from scipy.optimize import curve_fit
from lmfit import Model

from synthesizer.grid import Grid
from synthesizer.particle import Galaxy
from synthesizer.instruments.filters import FilterCollection

from synthesizer.emission_models import PacmanEmission
from synthesizer.emission_models.attenuation import PowerLaw
from synthesizer.emissions.utils import (
    Ha,
    Hb, 
    O2b,
    O2r,
    O3r,
    O3b,
    O3,
)
from synthesizer.conversions import lnu_to_absolute_mag

np.random.seed(680010)

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import cmasher as cmr

from utilities import (
    binned_weighted_quantile, 
    compute_metallicity_dust_correction,
    compute_metallicity_nodust_correction
)

from obs_data_plots import (
    plot_met_data,
    plot_rebels_data
)

cosmo = LambdaCDM(Om0=0.307, Ode0=0.693, H0=67.77, Ob0=0.04825)
plt.style.use('styles/style.mplstyle')

In [ ]:
def get_data(ii, tag, mstarlimit=1e8, OIIIlimit=1e-17*erg/s/cm**2, cosmo=cosmo):

    z = float(tag[5:].replace('p','.'))
    
    num = str(ii)

    if len(num) == 1:
        num =  '0'+num

    sim = "./data/flares_shared_data.hdf5"
    num = num+'/'

    with h5py.File(sim, 'r') as hf:
        Mstar   = np.array(hf[num+tag+'/Galaxy/Mstar_aperture'].get('30'), dtype = np.float32)*1e10
        MassweightedZ = np.array(hf[num+tag+'/Galaxy/Metallicity'].get('MassWeightedYoungStellarZ'), dtype = np.float64)
        MFUVatt = lnu_to_absolute_mag(np.array(hf[num+tag+'/Galaxy/BPASS_2.2.1/Chabrier300/Luminosity/DustModelI'].get('FUV'), dtype = np.float32) * erg / s / Hz)

        Av = np.array(hf[num+tag+'/Galaxy/BPASS_2.2.1/Chabrier300/Luminosity/DustModelI'].get('V'), dtype = np.float32) / np.array(hf[num+tag+'/Galaxy/BPASS_2.2.1/Chabrier300/Luminosity/Intrinsic'].get('V'), dtype = np.float32)
        Av = -2.5 * np.log10(Av)

        Halpha = np.array(hf[num+tag+'/Galaxy/BPASS_2.2.1/Chabrier300/Lines/DustModelI/HI6563'].get('Luminosity'), dtype = np.float64)
        Hbeta = np.array(hf[num+tag+'/Galaxy/BPASS_2.2.1/Chabrier300/Lines/DustModelI/HI4861'].get('Luminosity'), dtype = np.float64)
        HbetaEW = np.array(hf[num+tag+'/Galaxy/BPASS_2.2.1/Chabrier300/Lines/DustModelI/HI4861'].get('EW'), dtype = np.float64)
        Halphaint = np.array(hf[num+tag+'/Galaxy/BPASS_2.2.1/Chabrier300/Lines/Intrinsic/HI6563'].get('Luminosity'), dtype = np.float64)
        Hbetaint = np.array(hf[num+tag+'/Galaxy/BPASS_2.2.1/Chabrier300/Lines/Intrinsic/HI4861'].get('Luminosity'), dtype = np.float64)

        OIII5007 = np.array(hf[num+tag+'/Galaxy/BPASS_2.2.1/Chabrier300/Lines/DustModelI/OIII5007'].get('Luminosity'), dtype = np.float64)
        OIII4959 = np.array(hf[num+tag+'/Galaxy/BPASS_2.2.1/Chabrier300/Lines/DustModelI/OIII4959'].get('Luminosity'), dtype = np.float64)
        OII3727 = np.array(hf[num+tag+'/Galaxy/BPASS_2.2.1/Chabrier300/Lines/DustModelI/OII3726'].get('Luminosity'), dtype = np.float64) + np.array(hf[num+tag+'/Galaxy/BPASS_2.2.1/Chabrier300/Lines/DustModelI/OII3729'].get('Luminosity'), dtype = np.float64)
        NeIII3869 = np.array(hf[num+tag+'/Galaxy/BPASS_2.2.1/Chabrier300/Lines/DustModelI/NeIII3869'].get('Luminosity'), dtype = np.float64)
        # + np.array(hf[num+tag+'/Galaxy/BPASS_2.2.1/Chabrier300/Lines/DustModelI/HI3889'].get('Luminosity'), dtype = np.float64) + np.array(hf[num+tag+'/Galaxy/BPASS_2.2.1/Chabrier300/Lines/DustModelI/HeI3889'].get('Luminosity'), dtype = np.float64)

        OIII5007int = np.array(hf[num+tag+'/Galaxy/BPASS_2.2.1/Chabrier300/Lines/Intrinsic/OIII5007'].get('Luminosity'), dtype = np.float64)
        OIII4959int = np.array(hf[num+tag+'/Galaxy/BPASS_2.2.1/Chabrier300/Lines/Intrinsic/OIII4959'].get('Luminosity'), dtype = np.float64)
        OII3727int = np.array(hf[num+tag+'/Galaxy/BPASS_2.2.1/Chabrier300/Lines/Intrinsic/OII3726'].get('Luminosity'), dtype = np.float64) + np.array(hf[num+tag+'/Galaxy/BPASS_2.2.1/Chabrier300/Lines/Intrinsic/OII3729'].get('Luminosity'), dtype = np.float64)
        NeIII3869int = np.array(hf[num+tag+'/Galaxy/BPASS_2.2.1/Chabrier300/Lines/Intrinsic/NeIII3869'].get('Luminosity'), dtype = np.float64)

    lum_dist = cosmo.luminosity_distance(z).to("cm").value * cm
    OIII5007flux = OIII5007 * (erg/s) * (1+z) / (4 * np.pi * lum_dist**2)

    ok = (Halphaint>0) * (Mstar>mstarlimit) * (OIII5007flux>=OIIIlimit)

    return Mstar[ok], MFUVatt[ok], Halpha[ok], OIII5007[ok], OIII4959[ok], OII3727[ok], NeIII3869[ok], OIII5007int[ok], OIII4959int[ok], OII3727int[ok], NeIII3869int[ok], Hbeta[ok], Halphaint[ok], Hbetaint[ok], MassweightedZ[ok], Av[ok], HbetaEW[ok]

def plot_contour(x, y, ax, cmap, levels, label, zorder, alpha=0.5):
    
    xy = np.vstack([x, y])
    kde = gaussian_kde(xy)

    # Evaluate KDE on a grid
    xgrid = np.linspace(min(x), max(x), 100)
    ygrid = np.linspace(min(y), max(y), 100)
    Xgrid, Ygrid = np.meshgrid(xgrid, ygrid)
    Z = kde(np.vstack([Xgrid.ravel(), Ygrid.ravel()])).reshape(Xgrid.shape)
    
    ax.contour(Xgrid, Ygrid, Z, levels=levels, cmap=cmap, label=label, zorder=zorder, alpha=alpha)
    
    return ax

def plot_hexbin(x, y, ax, cmap, label, gridsize, zorder):
    
    ax.hexbin(x, y, gridsize=gridsize, cmap=cmap, label=label, zorder=zorder, mincnt=1, bins='log')
    
    return ax    

In [ ]:
# Define the grid
grid_name = "bpass-2.2.1-bin_chabrier03-0.1,300.0_cloudy-c23.01-sps"
grid_dir = "./data/"
grid = Grid(grid_name, grid_dir=grid_dir)

Zsun = 0.014
req_lines = {'H 1 6562.80A': 'Halpha',
             'H 1 4861.32A': 'Hbeta',
             'O 2 3726.03A,O 2 3728.81A': '[OII]3727,29',
             'O 3 4958.91A': '[OIII]4959',
             'O 3 5006.84A': '[OIII]5007',
             'Ne 3 3868.76A': 'NeIII3869'
}
tophats = {
    "UV1500": {"lam_eff": 1500*Angstrom, "lam_fwhm": 300*Angstrom},
    "UV2800": {"lam_eff": 2800*Angstrom, "lam_fwhm": 300*Angstrom},
}

filters = FilterCollection(
    filter_codes=[f'Generic/Johnson.V'],
    tophat_dict=tophats,
    new_lam=grid.lam
)

li = [
       Ha,
       Hb, 
       O2b,
       O2r,
       O3r,
       O3b,
       O3,
       'Ne 3 3868.76A'
   ]

In [ ]:
redshift = 6.
# Number of star particles
n = 100
ages = 10**np.random.uniform(0, np.log10(50), size=n) * Myr
Z = 10**np.random.uniform(-3.5, -2., size=n)
mass = 1e6*np.random.uniform(2, 5, n) * Msun 

# Dust distributions
max_tau = 7
mu, sigma = 0.1, 0.2
nsigma = np.arange(0,max_tau,2)
num_gals = len(nsigma)


# Property holders
logOH = np.zeros(4)
lum_int = np.zeros(4, dtype=object)
line_int = np.zeros(4, dtype=object)
sed_int = np.zeros(4, dtype=object)

line_emergent = np.zeros((4, num_gals), dtype=object)
sed_emergent = np.zeros((4, num_gals), dtype=object)
lum_att = np.zeros((4, num_gals))
Av = np.zeros((4, num_gals))
att_curve = np.zeros((4, num_gals, len(grid.lam)))

# Vary everything
gal = Galaxy(redshift=redshift)
gal.load_stars(
    ages=ages,
    metallicities=Z,
    initial_masses=mass 
)
ok = gal.stars.ages<=10*Myr
Zmean = np.sum(gal.stars.initial_masses[ok] * gal.stars.metallicities[ok]) / np.sum(gal.stars.initial_masses[ok])
Agemean = np.sum(gal.stars.initial_masses[ok] * gal.stars.ages[ok]) / np.sum(gal.stars.initial_masses[ok])

print (Zmean, Agemean)

# Keep everything fixed
gal_const = Galaxy(redshift=redshift)
gal_const.load_stars(
    ages=np.ones(n) * Agemean,
    metallicities=np.ones(n) * Zmean,
    initial_masses=mass 
)

# Vary age
gal_vary_age = Galaxy(redshift=redshift)
gal_vary_age.load_stars(
    ages=ages,
    metallicities=np.ones(n) * Zmean,
    initial_masses=mass 
)

# Vary metallicity
gal_vary_Z = Galaxy(redshift=redshift)
gal_vary_Z.load_stars(
    ages=np.ones(n) * Agemean,
    metallicities=Z,
    initial_masses=mass 
)

all_gals = [gal_const, gal_vary_age, gal_vary_Z, gal]

tau_v = np.zeros((num_gals, n))
for ii in range(num_gals):
    tmp  = mu + np.random.normal(0, nsigma[ii], n)
    tmp[tmp<0] = 0.01
    tau_v[ii] = tmp

for jj, kk in enumerate(all_gals):

    print ("Galaxy number: ", jj)
    ok = kk.stars.ages<=10*Myr
    Zmean = np.sum(kk.stars.initial_masses[ok] * kk.stars.metallicities[ok]) / np.sum(kk.stars.initial_masses[ok])    
    logOH[jj] = 8.69 + np.log10(Zmean/Zsun)

    for ii in range(num_gals):
        
        emodel = PacmanEmission(
            grid=grid,
            tau_v=tau_v[ii],
            dust_curve=PowerLaw(),
            per_particle=True
        )    
    
        kk.stars.get_spectra(emission_model=emodel)    
        kk.stars.get_lines(li,
            emodel
        )        
        
        line_emergent[jj][ii] = kk.stars.lines['emergent']
        kk.stars.spectra['emergent'].get_fnu(cosmo=cosmo, z=redshift)   
        tmp = kk.stars.spectra['emergent'].get_photo_lnu(filters, verbose=False)
        lum_att[jj][ii] = tmp['Generic/Johnson.V']
        
        sed_emergent[jj][ii] = kk.stars.spectra['emergent']
        att_curve[jj][ii] = -2.5 * np.log10(kk.stars.spectra['emergent'].luminosity/kk.stars.spectra['intrinsic'].luminosity)

    line_int[jj] = kk.stars.lines['intrinsic']    
    lum_int[jj] = kk.stars.spectra['intrinsic'].get_photo_lnu(filters, verbose=False)['Generic/Johnson.V']    
    sed_int[jj] = kk.stars.spectra['intrinsic']

    Av[jj] = -2.5 * np.log10(lum_att[jj]/lum_int[jj])
    
print (Av)
print (logOH)
print (np.log10(np.sum(mass)))

# Line ratios

<!-- $S2 = [SII]\lambda 6716,31 / H_{\alpha}$ -->

$R3 = [OIII]\lambda 5007/ H_{\beta}$

$R2 = [OII]\lambda 3727,29/H_{\beta}$

$R23 = \dfrac{[OIII]\lambda 4959,5007 + [OII]\lambda 3727,29}{H_{\beta}}$

$O32 = [OIII]\lambda5007/[OII]\lambda3727,29$

$Ne3O2 = [NeIII]\lambda3869/[OII]\lambda 3727,29$

In [ ]:
line_gals_int, line_gals_att = {}, {}

for jj, kk in enumerate(all_gals):
    
    tmp = {value: line_int[jj][key].luminosity.sum() for key, value in req_lines.items()}
    tmp['R23'] = (tmp['[OIII]4959']+tmp['[OIII]5007']+tmp['[OII]3727,29'])/tmp['Hbeta']
    tmp['R3'] = tmp['[OIII]5007']/tmp["Hbeta"]
    tmp['Ne3O2'] = tmp['NeIII3869']/tmp['[OII]3727,29']
    tmp['O32'] = tmp['[OIII]5007']/tmp['[OII]3727,29']
    
    if jj==0:
        line_gals_int = {jj: tmp}    
    else:
        line_gals_int[jj] = tmp   

    for ii in range(num_gals):
        
        tmp = {value: line_emergent[jj][ii][key].luminosity.sum() for key, value in req_lines.items()}
        tmp['R23'] = (tmp['[OIII]4959']+tmp['[OIII]5007']+tmp['[OII]3727,29'])/tmp['Hbeta']
        tmp['R3'] = tmp['[OIII]5007']/tmp["Hbeta"]
        tmp['Ne3O2'] = tmp['NeIII3869']/tmp['[OII]3727,29']
        tmp['O32'] = tmp['[OIII]5007']/tmp['[OII]3727,29']
        tmp['R2'] = tmp['[OII]3727,29']/tmp["Hbeta"]
        if ii==0:
            line_gals_att[jj] = {ii: tmp}    
        else:
            line_gals_att[jj][ii] = tmp        


In [ ]:
fig, axs = plt.subplots(nrows = 2, ncols = 2, figsize=(7, 7), facecolor='w', edgecolor='k', sharex=True, sharey=True)
axs = axs.ravel()
twinaxs = [ax.twinx() for ax in axs]

markers = ['s', 'H', '*', 'o']
custom = [Line2D([], [], marker=ii, markersize=8, color='dodgerblue', linestyle='None') for ii in markers]

for ii in range(4):
    if ii==0:
        axs[ii].axhline(y=logOH[ii], label='Mass-weighted', color='black', ls='dashed')
    else:
        axs[ii].axhline(y=logOH[ii], color='black', ls='dashed')


# Varying only Av
axs[0].scatter(0.0, compute_metallicity_dust_correction(line_gals_int[0], Balmerdust=False, Sanders=False, Curti=True)[0], color='black', s=50, marker=markers[0], label='Intrinsic')

# Varying Av and ages
axs[1].scatter(0.0, compute_metallicity_dust_correction(line_gals_int[1], Balmerdust=False, Sanders=False, Curti=True)[0], color='black', s=50, marker=markers[1])

# Varying Av and Z
axs[2].scatter(0.0, compute_metallicity_dust_correction(line_gals_int[2], Balmerdust=False, Sanders=False, Curti=True)[0], color='black', s=50, marker=markers[2])

# Varying all
axs[3].scatter(0.0, compute_metallicity_dust_correction(line_gals_int[3], Balmerdust=False, Sanders=False, Curti=True)[0], color='black', s=50, marker=markers[3])



for jj in range(4):

    for ii in range(num_gals):
        color = plt.cm.tab10(ii)
        
        axs[jj].scatter(Av[jj][ii], compute_metallicity_dust_correction(line_gals_att[jj][ii], Sanders=False, Curti=True)[0], edgecolor=color, facecolor='None', s=80, marker=markers[jj])  

        if jj==1:
            axs[jj].scatter(Av[jj][ii], compute_metallicity_dust_correction(line_gals_att[jj][ii], Balmerdust=False, Avdust=True, Av=Av[jj][ii], Sanders=False, Curti=True)[0], color=color, s=40, marker=markers[jj], alpha=0.5, label=rF'$\sigma={nsigma[ii]}$') 
        else:
            axs[jj].scatter(Av[jj][ii], compute_metallicity_dust_correction(line_gals_att[jj][ii], Balmerdust=False, Avdust=True, Av=Av[jj][ii], Sanders=False, Curti=True)[0], color=color, s=40, marker=markers[jj], alpha=0.5)             


for ax in axs:
    ax.grid(ls='dotted', alpha=0.5)
    ax.set_xticks(np.arange(0.1,1,0.2))
    ax.set_xlim(-0.03,0.8)
    # ax.set_ylim(7.01, 9.1)

for ax in twinaxs:
    ax.set_xlim(-0.03,0.8)
    # ax.set_ylim(7.01, 9.1)
    ax.set_yticks([])
    ax.set_yticklabels([])

axs[-1].set_xlabel(r'A$_{V}$', fontsize=14)
axs[-2].set_xlabel(r'A$_{V}$', fontsize=14)


twinaxs[0].legend([custom[0]], [r'Varying only A$_{\mathrm{V}}$'], loc='lower right', framealpha=0.2)
axs[0].legend(frameon=True, loc='center left', framealpha=0.2)

twinaxs[1].legend([custom[1]], ['Varying A$_{\mathrm{V}}$ and ages'], loc='upper left', framealpha=0.2)
axs[1].legend(frameon=True, loc='lower right', ncols=2, framealpha=0.2)

twinaxs[2].legend([custom[2]], ['Varying A$_{\mathrm{V}}$ and Z'], loc='upper right', framealpha=0.2)

twinaxs[3].legend([custom[3]], ['Varying all'], loc='lower right', framealpha=0.2)

fig.text(0.05, 0.5, r'$12+$log$_{10}$(O/H)', va='center', fontsize=14, rotation='vertical')

# fig.tight_layout()
fig.subplots_adjust(hspace=0,wspace=0.)

plt.savefig('plots/toy_met_Av.png', dpi=300)
plt.show()   

In [ ]:
def flares_galaxies(ii, func, data):
    
    Halpha, Hbeta, OII3727, OIII4959, OIII5007, NeIII3869, Av, HbetaEW = data
    
    gal = {}
    gal['Av'] = Av[ii]
    gal['HbetaEW'] = HbetaEW
    gal['Halpha'] = Halpha[ii]
    gal['Hbeta'] = Hbeta[ii]
    gal['[OII]3727,29'] = OII3727[ii]
    gal['[OIII]4959'] = OIII4959[ii]
    gal['[OIII]5007'] = OIII5007[ii]
    gal[ 'NeIII3869'] = NeIII3869[ii]
    
    gal['R23'] = (gal['[OIII]4959']+gal['[OIII]5007']+gal['[OII]3727,29'])/gal['Hbeta']
    gal['R3'] = gal['[OIII]5007']/gal["Hbeta"]
    gal['R2'] = gal['[OII]3727,29']/gal["Hbeta"]
    gal['Ne3O2'] = gal['NeIII3869']/gal['[OII]3727,29']
    gal['O32'] = gal['[OIII]5007']/gal['[OII]3727,29']
    
    out = func(gal)
    
    return out

def get_flares_galZ(data_att, data_int, tot_gals):    
    
    f = partial(compute_metallicity_dust_correction, Avdust=False, Balmerdust=True, Sanders=False, Curti=True)
    flares_dust_corr_Z = np.array([flares_galaxies(ii, f, data_att) for ii in range(tot_gals)])[:,0]
    
    f = partial(compute_metallicity_dust_correction, Avdust=True, Balmerdust=False, Sanders=False, Curti=True)
    flares_Avdust_corr_Z = np.array([flares_galaxies(ii, f, data_att) for ii in range(tot_gals)])[:,0]
    
    
    f = partial(compute_metallicity_dust_correction, Balmerdust=False, Sanders=False, Curti=True)
    flares_int_dust_corr_Z = np.array([flares_galaxies(ii, f, data_int) for ii in range(tot_gals)])[:,0]

    return np.asarray(flares_dust_corr_Z, dtype=np.float32), np.asarray(flares_Avdust_corr_Z, dtype=np.float32), np.asarray(flares_int_dust_corr_Z, dtype=np.float32)

In [ ]:
tags = ['010_z005p000', '009_z006p000', '008_z007p000', '007_z008p000', '006_z009p000', '005_z010p000']

df = pd.read_csv('./data/weights_grid.txt')
weights = np.array(df['weights'])

quantiles = [0.01,0.16,0.50,0.84,0.99]
OIIIlimit = 3e-18 * erg/s/cm**2

mbins = np.arange(8.,11.5,0.3)
mbincen = (mbins[1:] + mbins[:-1])/2.

lbins = -np.arange(17., 24.5, 0.5)[::-1]
lbincen = (lbins[1:] + lbins[:-1])/2.

data_lines = {}
data_mass = {}
data_luv = {}
met_data = {}

for ii, tag in enumerate(tags):

    z = float(tag[5:].replace('p','.'))

    func    = partial(get_data, tag=tag, OIIIlimit=OIIIlimit, cosmo=cosmo)
    pool    = MultiPool(processes=8)
    dat     = np.array(list(pool.map(func, np.arange(0,40))), dtype='object')
    pool.close()


    Mstar = np.log10(np.concatenate(dat[:,0]))
    MFUVatt = np.concatenate(dat[:,1])
    Halpha = np.concatenate(dat[:,2])
    OIII5007 = np.concatenate(dat[:,3])
    OIII4959 = np.concatenate(dat[:,4])
    OII3727 = np.concatenate(dat[:,5])
    NeIII3869 = np.concatenate(dat[:,6])
    OIII5007int = np.concatenate(dat[:,7])
    OIII4959int = np.concatenate(dat[:,8])
    OII3727int = np.concatenate(dat[:,9])
    NeIII3869int = np.concatenate(dat[:,10])
    Hbeta = np.concatenate(dat[:,11])
    Halphaint = np.concatenate(dat[:,12])
    Hbetaint = np.concatenate(dat[:,13])
    MassweightedZ = 8.69 + np.log10(np.concatenate(dat[:,14])/Zsun)
    Av = np.concatenate(dat[:,15])
    HbetaEW = np.concatenate(dat[:,16])

    tot_gals = len(Mstar)

    data_att = (Halpha, Hbeta, OII3727, OIII4959, OIII5007, NeIII3869, Av, HbetaEW)
    data_int = (Halphaint, Hbetaint, OII3727int, OIII4959int, OIII5007int, NeIII3869int, Av, HbetaEW)
    
    data_lines[F'R23_z{z}'] = (OIII5007+OIII4959+OII3727)/Hbeta
    data_lines[F'R3_z{z}'] = OIII5007/Hbeta
    data_lines[F'O32_z{z}'] = OIII5007/OII3727
    data_lines[F'Ne3O2_z{z}'] = NeIII3869/OII3727
    
    data_lines[F'intR23_z{z}'] = (OIII5007int+OIII4959int+OII3727int)/Hbetaint
    data_lines[F'intR3_z{z}'] = OIII5007int/Hbetaint
    data_lines[F'intO32_z{z}'] = OIII5007int/OII3727int
    data_lines[F'intNe3O2_z{z}'] = NeIII3869int/OII3727int

    
    flares_dust_corr_Z, flares_Avdust_corr_Z, flares_int_dust_corr_Z = get_flares_galZ(data_att, data_int, tot_gals)

    met_data[F'dustcorr_z{z}'] = flares_dust_corr_Z
    met_data[F'avdustcorr_z{z}'] = flares_Avdust_corr_Z
    met_data[F'intdustcorr_z{z}'] = flares_int_dust_corr_Z
    met_data[F'massweighted_z{z}'] = MassweightedZ
    met_data[F'mstar_z{z}'] = Mstar

    ws = np.zeros(tot_gals)
    n = 0
    for jj in range(40):
        if jj==0:
            ws[0:len(dat[jj][0])] = weights[jj]
        else:
            ws[n:n+len(dat[jj][0])] = weights[jj]

        n+=len(dat[jj][0])
    
    # Mass bins    
    hist, edges = np.histogram(Mstar, mbins)
    data_mass[F'N_z{z}'] = hist
    
    tmp = binned_weighted_quantile(Mstar, MassweightedZ, ws, mbins, quantiles)
    data_mass[F'massweighted_z{z}'] = tmp.T    
    
    tmp = binned_weighted_quantile(Mstar, flares_int_dust_corr_Z, ws, mbins, quantiles)
    data_mass[F'intdustcorr_z{z}'] = tmp.T
    tmp = binned_weighted_quantile(Mstar, flares_dust_corr_Z, ws, mbins, quantiles)
    data_mass[F'dustcorr_z{z}'] = tmp.T
    tmp = binned_weighted_quantile(Mstar, flares_Avdust_corr_Z, ws, mbins, quantiles)
    data_mass[F'avdustcorr_z{z}'] = tmp.T
    

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize=(5, 5), sharex=False, sharey=True, facecolor='w', edgecolor='k')
twinax = ax.twinx()

labels = ['Intrinsic (mass-weighted)', 'Intrinsic (line ratio)', 'Balmer corr', 'Av corr']

colors = ['black', 'green', 'indigo', 'red']

# z = 6
tag = tags[1]
z = float(tag[5:].replace('p','.'))

# Mass bins: R23-O32
ok = np.where(data_mass[F'N_z{z}']>5)[0]

ax.plot(mbincen[ok], data_mass[F'massweighted_z{z}'][2][ok], color=colors[0], label=labels[0])
ax.fill_between(mbincen[ok], data_mass[F'massweighted_z{z}'][1][ok], data_mass[F'massweighted_z{z}'][3][ok], color=colors[0], alpha=0.2)

ax.plot(mbincen[ok], data_mass[F'intdustcorr_z{z}'][2][ok], color=colors[1], label=labels[1])
ax.fill_between(mbincen[ok], data_mass[F'intdustcorr_z{z}'][1][ok], data_mass[F'intdustcorr_z{z}'][3][ok], color=colors[1], alpha=0.2)

ax.plot(mbincen[ok], data_mass[F'dustcorr_z{z}'][2][ok], color=colors[2], label=labels[2])
ax.fill_between(mbincen[ok], data_mass[F'dustcorr_z{z}'][1][ok], data_mass[F'dustcorr_z{z}'][3][ok], color=colors[2], alpha=0.2)

ax.plot(mbincen[ok], data_mass[F'avdustcorr_z{z}'][2][ok], color=colors[3], label=labels[3])
ax.fill_between(mbincen[ok], data_mass[F'avdustcorr_z{z}'][1][ok], data_mass[F'avdustcorr_z{z}'][3][ok], color=colors[3], alpha=0.2)


ax.plot(mbins[mbins<9.5], 0.11 * (mbins[mbins<9.5]-8) + 7.65, ls='dashed', label='Curti+2024 (Fit, $z=6-10$)', color='orange', lw=2)

ax.set_xlim(8, 11.)

ax.grid(ls='dashed')
twinax.set_yticks([])
twinax.set_yticklabels([])    
ax.grid(ls='dotted')    
ax.set_ylim(7.2, 9.2)
for label in (ax.get_xticklabels() + ax.get_yticklabels()):
    label.set_fontsize(12)

ax.legend(fontsize=11, frameon=True, framealpha=0.2, loc='upper left')
# twinax.legend(loc='upper left', fontsize=11, frameon=True, framealpha=0.5)
    
ax.set_ylabel('12+log(O/H)', fontsize=15)      
ax.set_xlabel('log$_{10}$(M$_{\star}$/M$_{\odot}$)', fontsize=15)

    
fig.subplots_adjust(hspace=0.2,wspace=0)
plt.savefig(F'plots/line_ratio_mzr_z{int(z)}.png', dpi=300, bbox_inches='tight')

In [ ]:
#Metallicity fit
def mzr_fit_fn(mstar, beta, Zm8):
    
    return beta * (mstar-8) + Zm8

def fit_mzr(mstar, met):
    
    model = Model(mzr_fit_fn)

    model.set_param_hint('beta', min=0.0, max=1)
    model.set_param_hint('Zm8', min=6.5, max=8.5)

    result = model.fit(met, mstar=mstar, beta=0.1, Zm8=7.5, method='emcee', fit_kws={'steps':4000, 'burn':2000, 'nwalkers':200, 'thin':20})
    
    return result

z = 6.0
ok = np.where(met_data[F'mstar_z{z}']<=9.5)[0]
for val in [F'dustcorr_z{z}', F'avdustcorr_z{z}', F'intdustcorr_z{z}', F'massweighted_z{z}']:

    result = fit_mzr(met_data[F'mstar_z{z}'][ok], met_data[val][ok])
    print (val, result.fit_report())

In [ ]:
# Redshift evolution of the MZR
fig, axs = plt.subplots(nrows = 1, ncols = 3, figsize=(9, 3), sharex=True, sharey=True, facecolor='w', edgecolor='k')
axs=axs.ravel()


labels = ['Intrinsic (line ratio)', 'Balmer corr', 'Av corr']

c_m =plt.get_cmap('cmr.bubblegum')
norm = matplotlib.colors.BoundaryNorm(np.arange(-0.5,6,1), c_m.N)
# create a ScalarMappable and initialize a data structure
s_m = matplotlib.cm.ScalarMappable(cmap=c_m, norm=norm)
s_m.set_array([])

for ii, tag in enumerate(tags):
    
    z = float(tag[5:].replace('p','.'))
    
    ok = np.where(data_mass[F'N_z{z}']>5)[0]
    
    
    axs[0].plot(mbincen[ok], data_mass[F'intdustcorr_z{z}'][2][ok], color=s_m.to_rgba(ii), lw=2, zorder=10)
    axs[0].fill_between(mbincen[ok], data_mass[F'intdustcorr_z{z}'][1][ok], data_mass[F'intdustcorr_z{z}'][3][ok], color=s_m.to_rgba(ii), alpha=0.1, zorder=1)    
    
    
    axs[1].plot(mbincen[ok], data_mass[F'dustcorr_z{z}'][2][ok], color=s_m.to_rgba(ii), lw=2, zorder=10)
    axs[1].fill_between(mbincen[ok], data_mass[F'dustcorr_z{z}'][1][ok], data_mass[F'dustcorr_z{z}'][3][ok], color=s_m.to_rgba(ii), alpha=0.1, zorder=1)
    
    axs[2].plot(mbincen[ok], data_mass[F'avdustcorr_z{z}'][2][ok], color=s_m.to_rgba(ii), lw=2, zorder=10)
    axs[2].fill_between(mbincen[ok], data_mass[F'avdustcorr_z{z}'][1][ok], data_mass[F'avdustcorr_z{z}'][3][ok], color=s_m.to_rgba(ii), alpha=0.1, zorder=1)
    
for ii in range(3):
    axs[ii].legend(title=labels[ii], frameon=False, title_fontsize=12, loc='upper left')

    axs[ii].grid(ls='dotted')    
    # axs[jj].set_ylim(7.4, 8.5)
    for label in (axs[ii].get_xticklabels() + axs[ii].get_yticklabels()):
        label.set_fontsize(12)
    

axs[0].set_ylabel('12+log(O/H)', fontsize=14)  

axs[1].set_xlabel('log$_{10}$(M$_{\star}$/M$_{\odot}$)', fontsize=14)

cbaxes = fig.add_axes([0.92, 0.25, 0.02, 0.5])
cbar = fig.colorbar(s_m, cax=cbaxes)
cbar.set_label(r'$z$', fontsize = 18)
cbar.set_ticks(np.arange(6))
cbar.set_ticklabels(np.arange(5,11))
cbaxes.invert_yaxis()
for label in cbaxes.get_yticklabels():
    label.set_fontsize(13)

fig.subplots_adjust(hspace=0,wspace=0)    
plt.savefig(F'plots/line_ratio_mzr_z_evo.png', dpi=300, bbox_inches='tight')